# Multispectral Band Analysis

This notebook analyzes multispectral band metadata to identify differences in band naming and assignment across firmware versions.


In [32]:
# Set up Python path to find the toolbox module
import sys
import os
from pathlib import Path

# Get the current notebook's directory and add its parent to the path
notebook_dir = Path(os.getcwd())
sys.path.append(str(notebook_dir))

# Import the toolbox module
from toolbox import plots
import pandas as pd

# Check if the module has the required function
print("Available functions in plots module:", [f for f in dir(plots) if not f.startswith('_')])


Available functions in plots module: ['DEFAULT_FIELDS', 'Path', 'compare_band_assignments', 'create_band_table', 'datetime', 'extract_metadata', 'extract_multispec_analysis', 'extract_multispec_bands', 'extract_tif_metadata', 'filter_date', 'filter_plot', 'filter_year', 'info', 'json', 'monthly', 'os', 'parse_directory_structure', 'parse_plots', 'pd', 're', 'recent', 'subprocess', 'yearly']


## Step 1: Parse Plot Directory Structure


In [33]:
# Parse the directory structure - adjust path to your data location
base_path = r"D:\TERN-Dronescape"  # Adjust to your data path
df = plots.parse_directory_structure(base_path)

# Preview the initial DataFrame
print(f"Found {len(df)} plot visits")
df.head()


Found 208 plot visits


,plot_id,visit_date,full_path
0,NTABRT0001,2024-08-29,D:\TERN-Dronescape\NTABRT0001\20240829
1,NTABRT0002,2024-08-29,D:\TERN-Dronescape\NTABRT0002\20240829
2,NTABRT0003,2024-08-29,D:\TERN-Dronescape\NTABRT0003\20240829
3,NTABRT0004,2024-08-28,D:\TERN-Dronescape\NTABRT0004\20240828
4,NTABRT0005,2024-08-28,D:\TERN-Dronescape\NTABRT0005\20240828


## Step 2: Extract Multispectral Band Metadata

This step will extract metadata from all multispectral bands in each plot directory. This may take some time to complete depending on the number of plots and images.


In [48]:
# First, check if exiftool is installed and working
import subprocess

try:
    result = subprocess.run(['exiftool', '-ver'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"exiftool is installed, version: {result.stdout.strip()}")
    else:
        print("exiftool command failed with return code", result.returncode)
        print("Make sure exiftool is installed and in your PATH")
except Exception as e:
    print(f"Error running exiftool: {e}")
    print("You need to install exiftool for this script to work.")
    print("Visit https://exiftool.org/ for installation instructions")

### --- Split here --- ###

# Fields we want to extract
fields = ['FileName', 'BandName', 'CentralWavelength', 'CenterWavelength', 'WavelengthFWHM', 'Bandwidth', 'RigCameraIndex', 'RigRelativesReferenceRigCameraIndex', 'Software', 'SwVersion']

# Run the extraction with the updated fields list
raw_bands_df = plots.extract_multispec_analysis(df, fields=fields, max_band_number=11)

# Create a new DataFrame with adjusted band numbering
bands_df = raw_bands_df.copy()

# Rename columns from Band0_X to Band1_X, etc.
renamed_columns = {}
for col in raw_bands_df.columns:
    # Check if it's a band metadata column
    if col.startswith('Band') and '_' in col:
        band_part, field_part = col.split('_', 1)
        if band_part.startswith('Band'):
            # Extract the band number and increment it by 1
            band_num = int(band_part.replace('Band', ''))
            new_band_num = band_num
            new_col = f"Band{new_band_num}_{field_part}"
            renamed_columns[col] = new_col
        
# Rename the columns in the DataFrame
bands_df = bands_df.rename(columns=renamed_columns)

# Add a new column for each band with the file suffix (last 6 characters)
for idx, row in bands_df.iterrows():
    # Look through columns that contain band file names
    for col in bands_df.columns:
        if col.startswith('Band') and col.endswith('_FileName'):
            # Get band number from column name
            band_num = col.split('_')[0]
            # If we have a filename, extract the last 6 characters
            if pd.notna(row[col]):
                filename = row[col]
                # Extract the last 6 characters (likely includes the extension)
                file_suffix = filename[-6:] if len(filename) >= 6 else filename
                # Create a new column for the suffix
                suffix_col = f"{band_num}_FileSuffix"
                bands_df.loc[idx, suffix_col] = file_suffix

# Show the DataFrame columns to see what we collected
print(f"Columns in the result DataFrame:")
print(bands_df.columns.tolist())

# Preview the data
bands_df.head()


exiftool is installed, version: 12.84
Extracting multispectral band metadata for 208 plot visits...
Fields to extract: FileName, BandName, CentralWavelength, CenterWavelength, WavelengthFWHM, Bandwidth, RigCameraIndex, RigRelativesReferenceRigCameraIndex, Software, SwVersion
Looking for bands 1 through 11
Processing NTABRT0001/20240829 (1/208)
Searching for TIF files in: D:\TERN-Dronescape\NTABRT0001\20240829\imagery\multispec\level0_raw
No files with _N.tif pattern found, looking for any TIF files...
Found 11 TIF files
Successfully extracted metadata for 11 files
Found 11 bands with RigCameraIndex
Searching for TIF files in: D:\TERN-Dronescape\NTABRT0002\20240829\imagery\multispec\level0_raw
No files with _N.tif pattern found, looking for any TIF files...
Found 11 TIF files
Successfully extracted metadata for 11 files
Found 11 bands with RigCameraIndex
Searching for TIF files in: D:\TERN-Dronescape\NTABRT0003\20240829\imagery\multispec\level0_raw
No files with _N.tif pattern found, lo

,plot_id,visit_date,full_path,Software,SwVersion,Band0_FileName,Band0_BandName,Band0_CentralWavelength,Band0_CenterWavelength,Band0_WavelengthFWHM,...,Band9_FileSuffix,Band10_FileSuffix,Band1_FileSuffix,Band2_FileSuffix,Band3_FileSuffix,Band4_FileSuffix,Band5_FileSuffix,Band6_FileSuffix,Band7_FileSuffix,Band8_FileSuffix
0,NTABRT0001,2024-08-29,D:\TERN-Dronescape\NTABRT0001\20240829,v1.4.0,v1.2.4,IMG_0000_1.tif,Blue,475.0,475.0,32.0,...,10.tif,11.tif,_2.tif,_3.tif,_4.tif,_5.tif,_6.tif,_7.tif,_8.tif,_9.tif
1,NTABRT0002,2024-08-29,D:\TERN-Dronescape\NTABRT0002\20240829,v1.4.0,v1.2.4,IMG_0000_1.tif,Blue,475.0,475.0,32.0,...,10.tif,11.tif,_2.tif,_3.tif,_4.tif,_5.tif,_6.tif,_7.tif,_8.tif,_9.tif
2,NTABRT0003,2024-08-29,D:\TERN-Dronescape\NTABRT0003\20240829,v1.4.0,v1.2.4,IMG_0000_1.tif,Blue,475.0,475.0,32.0,...,10.tif,11.tif,_2.tif,_3.tif,_4.tif,_5.tif,_6.tif,_7.tif,_8.tif,_9.tif
3,NTABRT0004,2024-08-28,D:\TERN-Dronescape\NTABRT0004\20240828,v1.4.0,v1.2.4,IMG_0000_1.tif,Blue,475.0,475.0,32.0,...,10.tif,11.tif,_2.tif,_3.tif,_4.tif,_5.tif,_6.tif,_7.tif,_8.tif,_9.tif
4,NTABRT0005,2024-08-28,D:\TERN-Dronescape\NTABRT0005\20240828,v1.4.0,v1.2.4,IMG_0000_1.tif,Blue,475.0,475.0,32.0,...,10.tif,11.tif,_2.tif,_3.tif,_4.tif,_5.tif,_6.tif,_7.tif,_8.tif,_9.tif


## Step 3: Analyze Software Versions


In [58]:
# Count the occurrences of each software version
sw_versions = bands_df['Software'].value_counts()
print("Number of unique plots per firmware version:")
sw_versions


Number of unique plots per firmware version:


Software
v1.4.0    91
v1.4.5    85
v1.4.1    16
v7.5.0    12
v7.4.0     2
Name: count, dtype: int64

## Step 4: Compare Band Assignments Across Firmware Versions


In [50]:
# Compare band assignments across versions
comparison = plots.compare_band_assignments(bands_df)
# comparison


## Step 5: Create a Compact Band Assignment Table


In [51]:
# Create a compact table showing band names by firmware version
band_table = plots.create_band_table(bands_df)
band_table


RigCameraIndex,0,1,10,2,3,4,5,6,7,8,9
Software,,,,,,,,,,,
v1.4.0,Blue (475.0 nm),Green (560.0 nm),Red edge-740 (740.0 nm),Red (668.0 nm),NIR (842.0 nm),Red edge (717.0 nm),Panchro (634.5 nm),Blue-444 (444.0 nm),Green-531 (531.0 nm),Red-650 (650.0 nm),Red edge-705 (705.0 nm)
v1.4.1,Blue (475.0 nm),Green (560.0 nm),Red edge-740 (740.0 nm),Red (668.0 nm),NIR (842.0 nm),Red edge (717.0 nm),Panchro (634.5 nm),Blue-444 (444.0 nm),Green-531 (531.0 nm),Red-650 (650.0 nm),Red edge-705 (705.0 nm)
v1.4.5,Blue (475.0 nm),Green (560.0 nm),Red edge-705 (705.0 nm),Red (668.0 nm),NIR (842.0 nm),Red edge (717.0 nm),Panchro (634.5 nm),Blue-444 (444.0 nm),Green-531 (531.0 nm),Red edge-740 (740.0 nm),Red-650 (650.0 nm)
v7.4.0,Blue (475.0 nm),Green (560.0 nm),nan (nan nm),Red (668.0 nm),NIR (842.0 nm),Red edge (717.0 nm),nan (nan nm),nan (nan nm),nan (nan nm),nan (nan nm),nan (nan nm)
v7.5.0,Blue (475.0 nm),Green (560.0 nm),nan (nan nm),Red (668.0 nm),NIR (842.0 nm),Red edge (717.0 nm),Blue-444 (444.0 nm),Green-531 (531.0 nm),Red-650 (650.0 nm),Red edge-705 (705.0 nm),Red edge-740 (740.0 nm)


## Step 6: Find Specific Band Assignment Changes


In [52]:
# Identify where the same RigCameraIndex has different band names across versions
# Ignore specific software versions like v7.5.0
changed_assignments = []
ignore_versions = ['v7.5.0', 'v7.4.0']  # Add versions to ignore here

# Get unique RigCameraIndex values from the comparison DataFrame
band_indices = comparison['RigCameraIndex'].unique()

for idx in band_indices:
    # Filter out ignored software versions
    filtered_versions = comparison[(comparison['RigCameraIndex'] == idx) & 
                                  (~comparison['Software'].isin(ignore_versions))]
    
    unique_band_names = filtered_versions['BandName'].unique()
    
    if len(unique_band_names) > 1:
        print(f"RigCameraIndex {idx} has multiple band name assignments:")
        
        for _, row in filtered_versions.iterrows():
            # Get file name from bands_df for this RigCameraIndex and Software version
            version = row['Software']
            version_data = bands_df[bands_df['Software'] == version]
            
            # Find column name for the current band index
            filename_col = f"Band{idx}_FileName"
            
            # Get filename if it exists in the DataFrame
            if filename_col in version_data.columns:
                # Get the first non-null filename for this band and version
                example_file = version_data[filename_col].dropna().iloc[0] if not version_data[filename_col].dropna().empty else "Unknown"
                print(f"  Version {version}: {row['BandName']} ({row['CentralWavelength']} nm) - File: {example_file}")
            else:
                print(f"  Version {version}: {row['BandName']} ({row['CentralWavelength']} nm)")
        print()
        
        # Add to list of changes
        change_entry = {
            'RigCameraIndex': idx,
            'BandNames': list(unique_band_names),
            'Versions': list(filtered_versions['Software'])
        }
        
        # Add filenames from bands_df for each version
        file_names = []
        for version in filtered_versions['Software']:
            version_data = bands_df[bands_df['Software'] == version]
            filename_col = f"Band{idx}_FileName"
            
            if filename_col in version_data.columns:
                # Get the first non-null filename for this band and version
                example_file = version_data[filename_col].dropna().iloc[0] if not version_data[filename_col].dropna().empty else "Unknown"
                file_names.append(example_file)
            else:
                file_names.append("Unknown")
        
        change_entry['FileNames'] = file_names
        changed_assignments.append(change_entry)

RigCameraIndex 10 has multiple band name assignments:
  Version v1.4.0: Red edge-740 (740.0 nm) - File: IMG_0000_11.tif
  Version v1.4.1: Red edge-740 (740.0 nm) - File: IMG_0000_11.tif
  Version v1.4.5: Red edge-705 (705.0 nm) - File: IMG_0000_11.tif

RigCameraIndex 8 has multiple band name assignments:
  Version v1.4.0: Red-650 (650.0 nm) - File: IMG_0000_9.tif
  Version v1.4.1: Red-650 (650.0 nm) - File: IMG_0000_9.tif
  Version v1.4.5: Red edge-740 (740.0 nm) - File: IMG_0000_9.tif

RigCameraIndex 9 has multiple band name assignments:
  Version v1.4.0: Red edge-705 (705.0 nm) - File: IMG_0000_10.tif
  Version v1.4.1: Red edge-705 (705.0 nm) - File: IMG_0000_10.tif
  Version v1.4.5: Red-650 (650.0 nm) - File: IMG_0000_10.tif



## Step 7: Analyze Filename Patterns and Suffixes


In [53]:
# Analyze filename patterns for bands with changed assignments
print("Analyzing filename patterns for changed band assignments...\n")

for change in changed_assignments:
    idx = change['RigCameraIndex']
    band_names = change['BandNames']
    versions = change['Versions']
    file_names = change['FileNames']
    
    print(f"RigCameraIndex {idx}: {', '.join(band_names)}")
    
    # Compare filenames across versions
    unique_filenames = set(file_names)
    if len(unique_filenames) > 1:
        print(f"  Different filename patterns detected across versions:")
        for version, filename in zip(versions, file_names):
            print(f"    Version {version}: {filename}")
    else:
        print(f"  Same filename pattern across all versions: {next(iter(unique_filenames))}")
        
    # Look for file suffix patterns in the raw data
    suffix_patterns = {}
    for version in versions:
        version_data = bands_df[bands_df['Software'] == version]
        suffix_col = f"Band{idx}_FileSuffix"
        
        if suffix_col in version_data.columns:
            suffixes = version_data[suffix_col].dropna().unique()
            suffix_patterns[version] = list(suffixes) if len(suffixes) > 0 else ["Unknown"]
    
    # Check if there are different suffix patterns across versions
    all_suffixes = set()
    for suffixes in suffix_patterns.values():
        all_suffixes.update(suffixes)
        
    if len(all_suffixes) > 1 or len(suffix_patterns) > 0:
        print("  File suffix patterns:")
        for version, suffixes in suffix_patterns.items():
            print(f"    Version {version}: {', '.join(suffixes)}")
    else:
        print("  No file suffix patterns found")
    
    # Check for filename structure consistency
    filename_structures = {}
    for version in versions:
        version_data = bands_df[bands_df['Software'] == version]
        filename_col = f"Band{idx}_FileName"
        
        if filename_col in version_data.columns:
            filenames = version_data[filename_col].dropna()
            if not filenames.empty:
                # Extract the base part of the filename (before index)
                base_parts = set()
                for fname in filenames:
                    # Try to extract consistent parts of the filename before the index
                    parts = fname.split(f"_{idx}")
                    if len(parts) > 1:
                        base_parts.add(parts[0])
                
                filename_structures[version] = list(base_parts)
    
    if filename_structures:
        print("  Filename base structures:")
        for version, structures in filename_structures.items():
            print(f"    Version {version}: {', '.join(structures)}")
    
    print()


Analyzing filename patterns for changed band assignments...

RigCameraIndex 10: Red edge-740, Red edge-705
  Same filename pattern across all versions: IMG_0000_11.tif
  File suffix patterns:
    Version v1.4.0: 11.tif
    Version v1.4.1: 11.tif
    Version v1.4.5: 11.tif
  Filename base structures:
    Version v1.4.0: 
    Version v1.4.1: 
    Version v1.4.5: 

RigCameraIndex 8: Red-650, Red edge-740
  Same filename pattern across all versions: IMG_0000_9.tif
  File suffix patterns:
    Version v1.4.0: _9.tif
    Version v1.4.1: _9.tif
    Version v1.4.5: _9.tif
  Filename base structures:
    Version v1.4.0: 
    Version v1.4.1: 
    Version v1.4.5: 

RigCameraIndex 9: Red edge-705, Red-650
  Same filename pattern across all versions: IMG_0000_10.tif
  File suffix patterns:
    Version v1.4.0: 10.tif
    Version v1.4.1: 10.tif
    Version v1.4.5: 10.tif
  Filename base structures:
    Version v1.4.0: 
    Version v1.4.1: 
    Version v1.4.5: 



## Step 8: Get a list of plots with specific firmware version


In [66]:
# Get a list of plots with specific firmware version
def get_plots_with_version(df, version):
    return df[df['Software'] == version][['plot_id', 'visit_date']].sort_values('plot_id')

# Example - change 'v1.2.4' to the version you're interested in
target_version = bands_df['Software'].unique()[0]  # Just taking the first version as an example
print(f"Plots with firmware version {target_version}:")
# to_update = get_plots_with_version(bands_df, target_version)


Plots with firmware version v1.4.0:


In [90]:
target_dirs = to_update.plot_id.values.tolist()

In [92]:
target_dirs[1:5]

['NTABRT0002', 'NTABRT0003', 'NTABRT0004', 'NTABRT0005']

## Final Step:Export the results

In [26]:
# Export the comparison table to CSV
# comparison.to_csv('band_comparison.csv', index=False)

# Export the compact band table
# band_table.to_csv('band_table_by_version.csv')

# Export the bands_df
bands_df.to_clipboard()
# bands_df.to_csv('bands_df.csv', index=False)